# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
import torch
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from sklearn.model_selection import RepeatedStratifiedKFold
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash


# Load data

In [ ]:
path_data = "D:/YandexDisk/Work/bbd/immunology/002_central_vs_yakutia/classification"
path_configs = "D:/Work/bbs/notebooks/immunology/002_central_vs_yakutia/pt_configs"
data = pd.read_excel(f"{path_data}/data.xlsx", index_col=0)
feats = pd.read_excel(f"{path_data}/feats.xlsx", index_col=0).index.values.tolist()

test_split_id = 0

val_n_splits = 4
val_random_state = 1337
val_fold_id = 0

for fold_id in range(val_n_splits):
    data[f"Fold_{fold_id}"] = data[f"Split_{test_split_id}"]

stratify_cat_parts = {
    'Central': data.index[(data['Region'] == 'Central') & (data[f"Split_{test_split_id}"] == 'trn_val')].values,
    'Yakutia': data.index[(data['Region'] == 'Yakutia') & (data[f"Split_{test_split_id}"] == 'trn_val')].values,
}
for part, ids in stratify_cat_parts.items():
    print(f"{part}: {len(ids)}")
    con = data.loc[ids, 'Age'].values
    ptp = np.ptp(con)
    num_bins = 5
    bins = np.linspace(np.min(con) - 0.1 * ptp, np.max(con) + 0.1 * ptp, num_bins + 1)
    binned = np.digitize(con, bins) - 1
    unique, counts = np.unique(binned, return_counts=True)
    occ = dict(zip(unique, counts))
    k_fold = RepeatedStratifiedKFold(
        n_splits=val_n_splits,
        n_repeats=1,
        random_state=val_random_state
    )
    splits = k_fold.split(X=ids, y=binned, groups=binned)
    
    for fold_id, (ids_trn, ids_val) in enumerate(splits):
        data.loc[ids[ids_trn], f"Fold_{fold_id}"] = "trn"
        data.loc[ids[ids_val], f"Fold_{fold_id}"] = "val"
        
test = data.loc[data[f"Split_{test_split_id}"] == "tst", feats + ['Region']]
train_validation = data.loc[data[f"Split_{test_split_id}"] == "trn_val", feats + ['Region'] + [f"Fold_{i}" for i in range(val_n_splits)]]
train_only = data.loc[data[f"Fold_{val_fold_id}"] == "trn", feats + ['Region']]
validation_only = data.loc[data[f"Fold_{val_fold_id}"] == "val", feats + ['Region']]
cv_indexes = [
    (
        np.where(train_validation.index.isin(train_validation.index[train_validation[f"Fold_{i}"] == 'trn']))[0],
        np.where(train_validation.index.isin(train_validation.index[train_validation[f"Fold_{i}"] == 'val']))[0],
    )
    for i in range(val_n_splits)
]

# Simple TabularModel training

In [8]:
trainer_config = read_parse_config(f"{path_configs}/TrainerConfig.yaml", TrainerConfig)
trainer_config['checkpoints'] = 'valid_loss'
trainer_config['load_best'] = True
trainer_config['auto_lr_find'] = True

tabular_model = TabularModel(
    data_config=f"{path_configs}/DataConfig.yaml",
    model_config=f"{path_configs}/models/CategoryEmbeddingModelConfig.yaml",
    optimizer_config=f"{path_configs}/OptimizerConfig.yaml",
    trainer_config=trainer_config,
    verbose=True,
    suppress_lightning_logger=False
)

tabular_model.fit(
    train=train_only,
    validation=validation_only,
    # target_transform=[np.log, np.exp],
    # callbacks=[DeviceStatsMonitor()],
)

2024-04-11 04:58:30,462 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-04-11 04:58:30,494 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-04-11 04:58:30,497 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

2024-04-11 04:58:30,583 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: CategoryEmbeddingModel

...

2024-04-11 04:58:34,238 - {pytorch_tabular.tabular_model:643} - INFO - Suggested LR: 0.006918309709189364. For plot
and detailed analysis, use `find_learning_rate` method.

2024-04-11 04:58:34,240 - {pytorch_tabular.tabular_model:652} - INFO - Training Started

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                      | Params
---------------------------------------------------------------
0 | _backbone        | CategoryEmbeddingBackbone | 49.6 K
1 | _embedding_layer | Embedding1dLayer          | 64    
2 | head             | LinearHead                | 130   
3 | loss             | CrossEntropyLoss          | 0     
---------------------------------------------------------------
49.8 K    Trainable params
0         Non-trainable params
49.8 K    Total params
0.199     Total estimated model params size (MB)


2024-04-11 04:58:36,813 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-04-11 04:58:36,815 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

## Play with trained model

In [10]:
tabular_model.predict(test, progress_bar='rich')

Output()

,Central_probability,Yakutia_probability,prediction
Index,,,
L10,0.999515,0.000485,Central
F1-L9,0.949430,0.050570,Central
F1-L18,0.949235,0.050765,Central
MQ2,0.997707,0.002293,Central
MQ13,0.926011,0.073989,Central
...,...,...,...
DP-8,0.431097,0.568903,Yakutia
T-85,0.725689,0.274311,Central
T-25,0.435630,0.564370,Yakutia


In [9]:
tabular_model.evaluate(test, verbose=True, ckpt_path="best")

Restoring states from the checkpoint path at D:\YandexDisk\Work\bbd\immunology\002_central_vs_yakutia\classification\pytorch_tabular\classification-1_epoch=5-valid_loss=0.46.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at D:\YandexDisk\Work\bbd\immunology\002_central_vs_yakutia\classification\pytorch_tabular\classification-1_epoch=5-valid_loss=0.46.ckpt
C:\Users\user\anaconda3\envs\py311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8977272510528564     │
│        test_auroc         │    0.9476190209388733     │
│     test_cohen_kappa      │    0.7474489808082581     │
│       test_f1_score       │    0.8932405710220337     │
│         test_loss         │    0.35063597559928894    │
│      test_precision       │    0.9034373760223389     │
│        test_recall        │    0.8977272510528564     │
│     test_specificity      │    0.7998917102813721     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.35063597559928894,
  'test_accuracy': 0.8977272510528564,
  'test_f1_score': 0.8932405710220337,
  'test_precision': 0.9034373760223389,
  'test_recall': 0.8977272510528564,
  'test_specificity': 0.7998917102813721,
  'test_cohen_kappa': 0.7474489808082581,
  'test_auroc': 0.9476190209388733}]

In [ ]:
tabular_model.config['checkpoints_path']

In [ ]:
print(tabular_model.trainer.checkpoint_callback.best_model_path)

In [ ]:
tabular_model.summary()

In [ ]:
tabular_model.save_model(tabular_model.config['checkpoints_path'])

In [ ]:
tabular_model.save_config(tabular_model.config['checkpoints_path'])

In [ ]:
tabular_model = TabularModel.load_model(tabular_model.config['checkpoints_path'])